In [1]:
import numpy as np
from numba import jit,njit,prange
import sys
path_to_folder = "XXX" # IMPORTANT change this to your path folder
sys.path.append(path_to_folder)

## Auxiliary function

In [2]:
@njit
def factorial3(x):
    '''
    Computes x!
    x <- integer
    '''
    n = 1
    for i in range(2, x+1):
        n *= i
    return n

@njit
def combi3(q,applications):
    '''
    Computes applications choose q
    
    q <- integer
    applications <- integer
    '''
    n = 1
    for i in range(q-applications+1, q+1):
        n *= i
    return n/factorial3(applications)

## Main functions

In [3]:
@jit(nopython=True)
def optimal_assortment(state,discount,weight,applications,no_purchase,n_samples,rand_suppliers,rand_customers):
    '''
    Computes optimal bundle MNL assortment to show the applicant
    
    Arguments:
    state <- current cumulative weight over suppliers (vector)
    discount function <- MNL-based discounts (vector)
    applications <- number of applications made by customers (integer)
    no_purchase <- weight of the no purchase option (float)
    n_samples <- number of random samples to estimate the expected revenue of bundle MNL (integer)
    rand_suppliers <- n_supplier-lenght vector capturing the suppliers-specific multiplicative revenue weight 
    rand_customers <- T-by-n_suppliers binary matrix capturing the customer consideration set
    
    Returns:
    assortment of suppliers (vector)
    expected matching rate (float)
    
    '''
    #current supplier match rates
    current_load = np.divide(state, 1+ state)
    #seller-level rewards
    delta = np.divide(state+ weight, 1+ state+ weight) - current_load
    for q in prange(state.shape[0]):
        #loop computes the discounted immediate reward associated with each suppliers
        index_discount = int(current_load[q]*float(discount.shape[0]))
        delta[q] = delta[q]*(1-discount[index_discount])
        delta[q] = rand_customers[q]*rand_suppliers[q]*delta[q]
    #ordering over suppliers (revenue-ordered policy is optimal)        
    rank_suppliers = np.argsort(delta)[::-1]
    #initializes the expected revenue estimate and match rate estimate for each
    #revenue-ordered assortment
    rev = np.zeros(delta.shape[0])
    match_rate =  np.zeros(delta.shape[0])   
    for q in prange(applications, delta.shape[0]):
        avg_rev = 0
        for z1 in prange(n_samples):
            #samples a set of applications
            picked = np.random.choice(rank_suppliers[:q],applications,\
                                      replace = False)
            for z2 in picked: 
                avg_rev+= delta[z2]
        #average rewards from applications
        avg_rev = avg_rev/n_samples
        count_vec = combi3(q,applications)
        #computes the expected revenue (approximately) and match rate
        rev[q] = avg_rev*(count_vec/(no_purchase+count_vec))
        match_rate[q] = count_vec/(no_purchase+count_vec)        
    #picks the highest revenue assortment
    q = np.argmax(rev)
    return(rank_suppliers[:q],match_rate[q] )

In [5]:
@jit(nopython=True)
def run_simulations(b_eps,g_eps,T,g_rate,n_suppliers,applications,no_purchase,discount,n_samples,\
                    rand_arrival,rand_choice,rand_suppliers,rand_customers):
    '''
    Function that runs the simulation instance
    
    Args
    b_eps <- attractiveness of bad customers to suppliers  (float)
    g_eps <- attractiveness of good customers to suppliers (float)
    T <- number of customers (integer)
    g_rate <- ex-ante fraction of good customer (float)
    n_suppliers <- number of suppliers (integer)
    applications <- number of applications (integer)
    no_purchase <- weight of no-purchase option (float)
    discount <- MNL-based discount function (float vector)
    n_samples <- number of samples to estimate the bundle MNL function + conflicts (integer)
    rand_arrival <- random numbers that determine whether customers are good (float vector)
    rand_choice <- random numbers that determine whether customers' choices (float vector)
    rand_suppliers <-  suppliers-specific multiplicative revenue weight (n_supplier-lenght vector)
    rand_customers <- customer consideration set   (T-by-n_suppliers binary matrix )
    
    Returns
    average number of conflicts
    average number of matches
    '''
    #initialises the state of the system and the weighted matches from customers
    #to suppliers
    state = np.zeros(n_suppliers)
    matches = np.zeros((T,n_suppliers))
    print('\n')
    # loop for the random experiments regarding the arrivals and customer choices
    for t in range(T):   
        is_good = rand_arrival[t] < g_rate
        if is_good:
            weight = g_eps
        else:
            weight = b_eps
        assortment,m = optimal_assortment(state,discount,weight,\
                                            applications,no_purchase, n_samples,rand_suppliers,rand_customers[t])
        if rand_choice[t]<m:
            picked = np.random.choice(assortment,applications,\
                                      replace = False)
            for j in picked:
                matches[t,j] = weight
            state = state + matches[t,:]

    total_conflicts = 0
    total_payoff = 0
    total_s_payoff = 0
    
    # loop for the number of conflicts
    for z in prange(n_samples):
        rand_s_choice = np.random.rand(n_suppliers)
        conflicts = np.zeros(T)
        for j in prange(n_suppliers):
            cum_match = np.cumsum(np.divide(matches[:,j],1+np.sum(matches[:,j])))
            total_payoff += cum_match[-1]            
            total_s_payoff += rand_suppliers[j]*cum_match[-1]
            cust = np.argmax(cum_match>rand_s_choice[j])
            if cum_match[cust]> rand_s_choice[j]:
                conflicts[cust] = conflicts[cust] +1
        total_conflicts = total_conflicts + np.sum(np.maximum(conflicts,1)-1) 
        
    print(T,n_suppliers,g_rate,applications,total_conflicts/n_samples,total_payoff/n_samples,total_s_payoff/n_samples)
    return(total_conflicts/n_samples,total_payoff/n_samples,matches)

## Main script

In [6]:
#bad customer attractiveness
b_eps = 0.05
#good customer attractiveness
g_eps = 0.2
#arrival rates of customers
g_rate_list = [0.05,0.1,0.15,0.2]
#number of customers
T_list = [400]
#number of applications
k_list = [1,3,5,10,15,20]
#heterogeneity in suppliers (not used)
supplier_rate = 0.5
#number of suppliers
n_suppliers = 100 #heterogeneity in suppliers (not used)
#number of samples to estimate the model (not varied)
n_samples = 500
#no purchase attractiveness for customers (not varied)
no_purchase = 1
#discount functions
discounts = np.load("../outputs/discount_functions_updated.npy")[0,:]
#results array
results = np.zeros((len(g_rate_list),len(T_list),len(k_list),4))

for T in T_list:
    for g_rate in g_rate_list:
        for applications in k_list:
            rand_arrival = np.random.rand(T)
            rand_choice = np.random.rand(T) 
            rand_suppliers = np.random.rand(n_suppliers)         
            rand_suppliers = 1*(rand_suppliers>supplier_rate) + 1*(rand_suppliers<supplier_rate) #not used
            cs = [np.random.choice(range(n_suppliers),n_suppliers, replace = False) for t in range(T)] #not used
            rand_customers = np.zeros((T,n_suppliers)) #not used
            for t in range(T):
                rand_customers[t,cs[t]] = 1 #not used
            cfl,pay,m = run_simulations(b_eps,g_eps,T,g_rate,n_suppliers,applications,no_purchase,discounts\
                                        ,n_samples,rand_arrival,rand_choice,rand_suppliers,rand_customers)
            results[g_rate_list.index(g_rate),T_list.index(T),0] = cfl
            results[g_rate_list.index(g_rate),T_list.index(T),1] = pay
            results[g_rate_list.index(g_rate),T_list.index(T),2] = np.mean(m.sum(axis = 0))        
            results[g_rate_list.index(g_rate),T_list.index(T),3] = np.std(m.sum(axis = 0))        



400 100 1 0.0 18.783431544302385 18.783431544302385


400 100 3 1.868 40.906035141333184 40.906035141333184


400 100 5 3.754 53.74185594207065 53.74185594207065


400 100 10 6.352 69.13461538465036 69.13461538465036


400 100 15 9.234 78.60901395558194 78.60901395558194


400 100 20 9.77 82.51529457825234 82.51529457825234


400 100 1 0.0 20.78343508343747 20.78343508343747


400 100 3 2.11 42.82019967778456 42.82019967778456


400 100 5 4.296 55.87253198027143 55.87253198027143


400 100 10 7.802 72.74843473609809 72.74843473609809


400 100 15 8.614 77.9731379731575 77.9731379731575


400 100 20 10.59 83.46973056508084 83.46973056508084


400 100 1 0.0 22.395754564729017 22.395754564729017


400 100 3 2.86 47.64980646691029 47.64980646691029


400 100 5 4.656 57.47002051510658 57.47002051510658


400 100 10 8.982 75.17830105140565 75.17830105140565


400 100 15 10.538 81.38588000677191 81.38588000677191


400 100 20 11.682 85.29399042665962 85.29399042665962


400 100 1 0.0 23.683

In [7]:
#bad customer attractiveness
b_eps = 0.05
#good customer attractiveness
g_eps = 0.2
#arrival rates of customers
g_rate_list = [0.05]
#number of customers
T_list = [200,400,600,800]
#number of applications
k_list = [1,3,5,10,15,20]
#heterogeneity in suppliers (not used)
supplier_rate = 0.5
#number of suppliers
n_suppliers = 100 #heterogeneity in suppliers (not used)
#number of samples to estimate the model (not varied)
n_samples = 500
#no purchase attractiveness for customers (not varied)
no_purchase = 1
#discount functions
discounts = np.load("discount_functions_updated.npy")[0,:]
#results array
results = np.zeros((len(g_rate_list),len(T_list),len(k_list),4))

for T in T_list:
    for g_rate in g_rate_list:
        for applications in k_list:
            rand_arrival = np.random.rand(T)
            rand_choice = np.random.rand(T) 
            rand_suppliers = np.random.rand(n_suppliers)         
            rand_suppliers = 1*(rand_suppliers>supplier_rate) + 1*(rand_suppliers<supplier_rate) #not used
            cs = [np.random.choice(range(n_suppliers),n_suppliers, replace = False) for t in range(T)] #not used
            rand_customers = np.zeros((T,n_suppliers)) #not used
            for t in range(T):
                rand_customers[t,cs[t]] = 1 #not used
            cfl,pay,m = run_simulations(b_eps,g_eps,T,g_rate,n_suppliers,applications,no_purchase,discounts\
                                        ,n_samples,rand_arrival,rand_choice,rand_suppliers,rand_customers)
            results[g_rate_list.index(g_rate),T_list.index(T),0] = cfl
            results[g_rate_list.index(g_rate),T_list.index(T),1] = pay
            results[g_rate_list.index(g_rate),T_list.index(T),2] = np.mean(m.sum(axis = 0))        
            results[g_rate_list.index(g_rate),T_list.index(T),3] = np.std(m.sum(axis = 0))



200 100 1 0.0 10.02683982683443 10.02683982683443


200 100 3 1.506 26.08511922304687 26.08511922304687


200 100 5 3.258 36.74565867404425 36.74565867404425


200 100 10 6.78 52.14866434377139 52.14866434377139


200 100 15 9.3 61.71988388965234 61.71988388965234


200 100 20 12.258 69.12955504281237 69.12955504281237


400 100 1 0.0 18.0976217019714 18.0976217019714


400 100 3 1.716 40.267615973512626 40.267615973512626


400 100 5 3.282 52.26002018399916 52.26002018399916


400 100 10 6.288 68.98592659415282 68.98592659415282


400 100 15 9.038 78.39914983044343 78.39914983044343


400 100 20 9.788 82.60848523099399 82.60848523099399


600 100 1 0.0 25.009877478846423 25.009877478846423


600 100 3 1.674 50.26592513178441 50.26592513178441


600 100 5 3.27 62.9597139409227 62.9597139409227


600 100 10 6.228 78.46977190520921 78.46977190520921


600 100 15 6.696 83.86441122472745 83.86441122472745


600 100 20 7.632 87.48427672956052 87.48427672956052


800 100 1 0.0 30.156928480